In [ ]:
import collections
import json
import re



from spacy.lang.en import English
spacy_nlp = English()
spacy_nlp.add_pipe("sentencizer")

Comment = collections.namedtuple("Comment", "text sentences")
class Text(object):
    def __init__(self, text):
        sentence_texts = []
        sentence_indices = []
        for chunk in text.split("\n"):
            doc = spacy_nlp(chunk)
            for sent in doc.sents:
                sentence_text = sent.text.strip()
                if not sentence_text:
                    continue
                if sentence_indices:
                    offset = sentence_indices[-1][1]
                    index = offset + text[offset: ].find(sentence_text)
                else :
                    index = text.find(sentence_text)
                sentence_texts.append(sentence_text)
                sentence_indices.append((index, index + len(sentence_text)))
        assert len(sentence_texts) == len(sentence_indices)

        final_sentences = []
        for i in range(len(sentence_texts) - 1):
            start, end = sentence_indices[i]
            sentence_text = sentence_texts[i]
            next_sentence_start = sentence_indices[i + 1][0]
            suffix = "\n" * text[end: next_sentence_start].count("\n")
            assert sentence_text == text[start: end]
            final_sentences.append(make_sentence_dict(start, end, suffix))

        if sentence_indices:
            final_start, final_end = sentence_indices[-1]
            final_sentences.append(make_sentence_dict(final_start, final_end, ""))

        self.text = Comment(text, final_sentences)._asdict()

Before tokenization, convert all references into "Reference_n.". Convert all citations into "Citep_n." Remove any remaining et als.

maybe also grab urls? do they get properly tokenized?

In [ ]:
import json
from openreview_lib import DATASETS
for dataset in DATASETS:
    PAIR_FILE = "../../data/review_rebuttal_pair_dataset/" + dataset + ".json"
    if dataset == 'unstructured':
        continue
    with open(PAIR_FILE, 'r') as f:
        pair_obj = json.load(f)

In [ ]:
import re

DATE_MATCHER = re.compile("(\d){4}")
REFERENCE_MATCHER = re.compile("^\[..?\]")
TITLE_CASE_WORD = re.compile("[A-Z][a-z]+")
END_DATE_MATCHER = re.compile(".*[A-Z][a-z]+.*20(\d){2}.?$")
START_SQB = re.compile("^\[.*")

SHORT_PAREN_MATCHER = "\([^\(\)]\)"
SQB_MATCHER = "\[.??\]"
ETAL_MATCHER = "et\.?\sal\.?"
ARXIV = ".*arXiv.preprint.*"
ENDSWITH_DATE = re.compile(".*(\d){4}.?$")

placeholder_prefix = "$$$ABCD"
placeholder_suffix = "$$$"

def generate_placeholders():
    i=0
    while True:
        new_placeholder = placeholder_prefix + str(i) + placeholder_suffix
        i += 1
        yield new_placeholder
        
def substitute(line, match, gen):
    start, end = match.span()
    before, match_text, after = line[:start], line[start:end], line[end:]
    new_placeholder = next(gen)
    updated_line = before + new_placeholder + after
    return updated_line, {new_placeholder:match_text}

for pair in pair_obj["review_rebuttal_pairs"]:
    review_text = pair["review_text"]["text"]
    
    gen = generate_placeholders()
    placeholders = collections.OrderedDict()
    replaced_lines = ""
    for line in review_text.split("\n"):
        
        updated_line = line
        # Replace all occurrences of et al and misspellings
#         etal_matches = list(re.finditer(ETAL_MATCHER, line))
#         for match in reversed(sorted(etal_matches, key=lambda x:x.span()[0])):
#             if re.match("\s", line[match.start() - 1]):
#                 updated_line, mini_placeholder_map = substitute(updated_line, match, gen)
#                 placeholders.update(mini_placeholder_map)

        # Easier to search for references by searching for dates
        flag = False
        contains_date = re.findall(DATE_MATCHER, updated_line) # Really just checking for 4 digits
        if contains_date:
            start_sqb_match = re.match(START_SQB, updated_line)
            if start_sqb_match is not None:
                #updated_line, mini_placeholder_map = substitute(updated_line, start_sqb_match, gen)
                #placeholders.update(mini_placeholder_map)
                print("1\t"+line)
                flag = True
            else:
                maybe_arxiv_match = re.match(ARXIV, updated_line)
                if maybe_arxiv_match is not None:
#                     updated_line, mini_placeholder_map = substitute(updated_line, maybe_arxiv_match, gen)
#                     placeholders.update(mini_placeholder_map)
                    print("1\t"+line)
                    flag = True
                else:
                    namelike = re.findall(TITLE_CASE_WORD, updated_line)
                    if len(namelike) > 3 and len(updated_line) < 200 and re.match(
                    ENDSWITH_DATE, updated_line):
#                         updated_line, mini_placeholder_map = substitute(
#                             updated_line, 
#                             re.match(ENDSWITH_DATE, updated_line), gen)
#                 placeholders.update(mini_placeholder_map)
                
                        print("1\t"+line)
                flag = True
                
        if not flag:
            print("0\t"+line)


In [ ]:
k =spacy_nlp('The paper presents extensive, interesting results. I do want to point that they seem to be considerably off of the LibriSpeech state of the art, e.g. see K. Irie $$$ABCD1$$$ Interspeech 2019.')

In [ ]:
for i in k.sents:
    print(i)

In [ ]:
l = re.findall(ETAL_MATCHER, "Why not a direct comparison with the models proposed by Wang et al., 2019, He et al., 2019?")

In [ ]:
print("et al" == "et al")

for i in l:
    print(i)
    


In [29]:
import scispacy
import spacy

sci_nlp = spacy.load("en_core_sci_scibert")

doc = sci_nlp(difficult_passage.replace("?", "."))

for s in doc.sents:
    print(s)


However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.
In the mean time, the notion of "Neural Stethoscopes" could be much more generally applied.
Without applications in other domains, it is not immediately clear what the paper's implication is.



a) In the paper, the authors mentioned that "This simple metric is non-parametric and we found that the results are not sensitive to the selection of k".
Can authors provide more details, e.g., empirical results, about it.
What is its rationale.



I wonder if this tokenizer is just not able to tokenize on questions.
I would believe that, since there are rarely questions in scientific text. But wow, isn't that so irritating.


In this case, robust training framework can defend blind-spot attacks, right.
I agree with the authors that the generalization error is due to the mismatch between trai

In [27]:

from spacy.lang.en import English
spacy_nlp = English()
spacy_nlp.add_pipe("sentencizer")


doc = spacy_nlp(difficult_passage)

for s in doc.sents:
    print(s)




However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.
In the mean time, the notion of "Neural Stethoscopes" could be much more generally applied.
Without applications in other domains, it is not immediately clear what the paper's implication is.


a) In the paper, the authors mentioned that "This simple metric is non-parametric and we found that the results are not sensitive to the selection of k".
Can authors provide more details, e.g., empirical results, about it?
What is its rationale?


I wonder if this tokenizer is just not able to tokenize on questions?
I would believe that, since there are rarely questions in scientific text.
But wow, isn't that so irritating?


In this case, robust training framework can defend blind-spot attacks, right?
I agree with the authors that the generalization error is due to the mismatch between traini

In [ ]:
# Sandbox

current_rebuttal = """Thank you for your review and we would be delighted to address your concerns, but do require some clarifications.

While a learnable lambda could be considered we would argue that the learning of this parameter beyond the grid-search applied in the submission is somewhat tangential to our primary contribution: a unified framework which lends itself to targeted representation analysis and modification.

1)
The notion of a map of \lambdas sounds interesting. However, at present, it is not clear to us what this refers to as \lambda is a weighting on a loss term. Clarification would be much appreciated so we can fully engage with this point. As far as the existing approach is concerned, Figure 6 illustrates the influence of \lambda on the accuracy and correlation of global and local stability prediction.

2)
The inconsistent correlations between the two tasks are exactly the scenarios where stethoscopes come into their own: testing positive and negative regimes of lambda (corresponding to auxiliary and adversarial training, respectively) reveals the interplay between the two tasks and potentially allows for de-biasing the algorithm as shown in Figure 6a. Therefore, in contrast to the design not considering these relationships, it explicitly addresses them.

Could you please elaborate on the comment ’the current design […] simply sums them up’? The stethoscope module has its own trainable parameters and a separate loss function. Only the encoder shares weights between main and secondary task.
"""

In [ ]:


doc = spacy_nlp(current_rebuttal)

for s in doc.sents:
    print(s)


In [ ]:
sent1 = spacy_nlp(current_rebuttal)

In [11]:
difficult_passage = """However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all. In the mean time, the notion of "Neural Stethoscopes" could be much more generally applied. Without applications in other domains, it is not immediately clear what the paper's implication is.

a) In the paper, the authors mentioned that "This simple metric is non-parametric and we found that the results are not sensitive to the selection of k". Can authors provide more details, e.g., empirical results, about it? What is its rationale?

I wonder if this tokenizer is just not able to tokenize on questions? I would believe that, since there are rarely questions in scientific text. But wow, isn't that so irritating?

In this case, robust training framework can defend blind-spot attacks, right? I agree with the authors that the generalization error is due to the mismatch between training data and test data distribution, however, I am not convinced that blind-spot attacks are effective enough to robust training. 

b) In the paper, "We find that these blind-spots are prevalent and can be easily found without resorting to complex

generative models like in Song et al. (2018). 

- Ablation study (This has been described in detail in Tab. 1 and also in Sec. 3)
To empirically resolve the above concerns, it is necessary to present the empirical comparison with the "static" softmax.
Namely, the methods of HLN/HBN should be carefully compared to LN/BN of "alpha=4", not only those of alpha=16 shown in Table 1&2; the comparison in Table 3 seems unfair since the authors apply the static softmax without normalization.
And, it would be better to show the performance of heated-up softmax "without" normalization to show the important role of the normalization, as done in [Wang et al., 2017].
In summary, since the proposed method is composed of a heating-up approach and feature normalization, the authors are required to validate the method from those two aspects, respectively, for increasing the significance of this paper.

- Other loss function
For achieving a compactness in feature representation, the simple softmax requires both temperature and normalization. It, however, is also conceivable to employ the other types of loss function for that purpose, such as [a] which is based on the (Mahalanobis) distance with taking into account the margin between categories. The distance based loss also embeds features into localized clusters, which satisfies the authors' objective in this work. To validate the proposed method, it is required to compare the method with such a different types of loss function.

[a] Wan, W., Zhong, Y., Li, T., & Chen, J. (2018). Rethinking Feature Distribution for Loss Functions in Image Classification, In CVPR2018, pp. 9117–9126."""

In [25]:
import scispacy
import spacy
from spacy.lang.en import English
from termcolor import colored
import tabulate

sci_nlp = spacy.load("en_core_sci_scibert")
spacy_nlp = English()
spacy_nlp.add_pipe("sentencizer")


def tokenize_scispacy(text):
    return list(sci_nlp(text).sents)
def tokenize_spacy(text):
    return list(spacy_nlp(text).sents)


COLORS = ["red", "blue"]
def build_column(tokenized_sentences):
    column = []
    for i, sentence in enumerate(tokenized_sentences):
        color = COLORS[i%2]
        column += [colored(sentence, color)]
    return column

def build_tokenized_col(tokenize_fn, text):
    tokenized = tokenize_fn(text)
    return build_column(tokenized)

def make_cols(text):
    spacy_col = build_tokenized_col(tokenize_spacy, text)
    scispacy_col = build_tokenized_col(tokenize_scispacy, text)
    max_len = max([len(spacy_col), len(scispacy_col)])

    spacy_col += [""] * (max_len - len(spacy_col))
    scispacy_col += [""] * (max_len - len(scispacy_col))
    return spacy_col, scispacy_col


def make_table(text):
    return tabulate.tabulate(
        zip(*make_cols(text)),
        headers = ["Spacy", "Scispacy"])

for chunk in difficult_passage.split("\n"):
    if not chunk:
        continue
    print(make_table(difficult_passage))
    print()
    print()

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

...

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

Spacy                                                                                                                                                                                                                                       Scispacy
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all.               

In [ ]:
# if there is a year, use scispacy
# 

In [1]:
another_thing_with_questions = "Do you have a question? No, I have a comment."
another_thing_with_questions = """The introduction and the title does not match. Metric learning does not require to specify the dimension; while the embedding has to specify the reduced dimension. I feel confused that the authors mix these two concepts.

The objective in (1) is very close to that of t-SNE[5], where it uses the KL as the objective. Then other update formula are similar.  

This paper facilitates the effect of temperature in the Softmax function to heuristically learn a compact and spread-out embedding. However, such an idea have been widely used and investigated in Reinforcement learning [1], Knowledge distillation [2], classification [3] and discrete variable optimization [4] and t-SNE visualization [5] etc. Thus, the insight about the temperature effect on the embedding from the second last layer, cannot be novel any more. Based on this, the proposed ``heating-up” strategy to leverage its effect on the embedding is heuristic, since the temperature parameter is manually set instead of automatically learning. In this case, I do expect the authors should provide more in-depth theoretical analysis. 

The authors do not present more experimental results on the correlation between the final performance and this temperature setting. 

Besides, as the alpha increases or decreases, the side-effect on the learning rate setting for the optimization have not clearly analyzed, which leaves more concerns on tuning performance. 


[1] Sutton, R. S. and Barto A. G. Reinforcement Learning: An Introduction. The MIT Press, Cambridge, MA, 1998.
[2] Hinton G, Vinyals O, Dean J. Distilling the knowledge in a neural network. NIPS 2015.
[3] Guo, Chuan, et al. "On calibration of modern neural networks." ICML 2017.
[4] Jang E, Gu S, Poole B. Categorical reparameterization with gumbel-softmax. ICLR 2017.
[5] Maaten L, Hinton G. Visualizing data using t-SNE[J]. Journal of machine learning research, 2008, 9(Nov): 2579-2605."""

In [2]:
import scispacy
import spacy
from spacy.lang.en import English
from termcolor import colored
import tabulate

sci_nlp = spacy.load("en_core_sci_scibert")
spacy_nlp = English()
spacy_nlp.add_pipe("sentencizer")

#doc = spacy_nlp(another_thing_with_questions.replace("?","."))
#doc = spacy_nlp(another_thing_with_questions)
doc = sci_nlp(another_thing_with_questions)

for s in doc.sents:
#     print(type(s))
#     print(dir(s))
#     print(s.start, s.end)
    print(s)


The introduction and the title does not match.
Metric learning does not require to specify the dimension; while the embedding has to specify the reduced dimension.
I feel confused that the authors mix these two concepts.


The objective in (1) is very close to that of t-SNE[5], where it uses the KL as the objective.
Then other update formula are similar.
 

This paper facilitates the effect of temperature in the Softmax function to heuristically learn a compact and spread-out embedding.
However, such an idea have been widely used and investigated in Reinforcement learning [1], Knowledge distillation [2], classification [3] and discrete variable optimization [4] and t-SNE visualization [5] etc.
Thus, the insight about the temperature effect on the embedding from the second last layer, cannot be novel any more.
Based on this, the proposed ``heating-up” strategy to leverage its effect on the embedding is heuristic, since the temperature parameter is manually set instead of automatically l

In [21]:
text = """However, my concern is that the paper focuses only on a very specific application domain, and an improvement over the niche dataset with much more supervision (from the extension) is not surprising at all. In the mean time, the notion of "Neural Stethoscopes" could be much more generally applied. Without applications in other domains, it is not immediately clear what the paper's implication is.

a) In the paper, the authors mentioned that "This simple metric is non-parametric and we found that the results are not sensitive to the selection of k". Can authors provide more details, e.g., empirical results, about it? What is its rationale?

I wonder if this tokenizer is just not able to tokenize on questions? I would believe that, since there are rarely questions in scientific text. But wow, isn't that so irritating?

In this case, robust training framework can defend blind-spot attacks, right? I agree with the authors that the generalization error is due to the mismatch between training data and test data distribution, however, I am not convinced that blind-spot attacks are effective enough to robust training. 

b) In the paper, "We find that these blind-spots are prevalent and can be easily found without resorting to complex

generative models like in Song et al. (2018). 

- Ablation study (This has been described in detail in Tab. 1 and also in Sec. 3)
To empirically resolve the above concerns, it is necessary to present the empirical comparison with the "static" softmax.
Namely, the methods of HLN/HBN should be carefully compared to LN/BN of "alpha=4", not only those of alpha=16 shown in Table 1&2; the comparison in Table 3 seems unfair since the authors apply the static softmax without normalization.
And, it would be better to show the performance of heated-up softmax "without" normalization to show the important role of the normalization, as done in [Wang et al., 2017].
In summary, since the proposed method is composed of a heating-up approach and feature normalization, the authors are required to validate the method from those two aspects, respectively, for increasing the significance of this paper.

- Other loss function
For achieving a compactness in feature representation, the simple softmax requires both temperature and normalization. It, however, is also conceivable to employ the other types of loss function for that purpose, such as [a] which is based on the (Mahalanobis) distance with taking into account the margin between categories. The distance based loss also embeds features into localized clusters, which satisfies the authors' objective in this work. To validate the proposed method, it is required to compare the method with such a different types of loss function.

[a] Wan, W., Zhong, Y., Li, T., & Chen, J. (2018). Rethinking Feature Distribution for Loss Functions in Image Classification, In CVPR2018, pp. 9117–9126."""

In [50]:
import collections

def get_span_map(nlp_pipeline, text):
    span_map = collections.OrderedDict()
    doc = nlp_pipeline(text.replace("\n", " "))
    return [(s.start_char, s.end_char) for s in doc.sents]

normal_span_list = get_span_map(spacy_nlp, text)
sci_span_list = get_span_map(sci_nlp, text)

In [67]:
final_spans = collections.OrderedDict()

def get_last_index(span_list):
    return span_list[-1][1]

def get_contained_spans(super_span, span_list):
    super_start, super_end = super_span
    contained_spans = []
    for i, (start, end) in enumerate(span_list):
        if start >= super_start and end <= super_end:
            contained_spans.append(i)
    if not contained_spans:
        return None, None
    return contained_spans[0], contained_spans[-1]
        
# Finding a sci span that contains a bunch of normal spans. we need to merge the normal spans
normal_spans_to_merge = {}
for sci_span in sci_span_list:
    sci_start, sci_end = sci_span
    first_contained_span, last_contained_span = get_contained_spans(sci_span, normal_span_list)
    if first_contained_span is None:
        continue
    if last_contained_span - first_contained_span > 1:
        normal_spans_to_merge[first_contained_span] = last_contained_span
        
print(normal_spans_to_merge)

for start_span, end_span in normal_spans_to_merge.items():
    print(text[normal_span_list[start_span][0]:normal_span_list[end_span][1]])
    print("*" * 80)
    


{7: 10, 16: 18, 24: 26}
I would believe that, since there are rarely questions in scientific text. But wow, isn't that so irritating?

In this case, robust training framework can defend blind-spot attacks, right? I agree with the authors that the generalization error is due to the mismatch between training data and test data distribution, however, I am not convinced that blind-spot attacks are effective enough to robust training.
********************************************************************************
Namely, the methods of HLN/HBN should be carefully compared to LN/BN of "alpha=4", not only those of alpha=16 shown in Table 1&2; the comparison in Table 3 seems unfair since the authors apply the static softmax without normalization.
And, it would be better to show the performance of heated-up softmax "without" normalization to show the important role of the normalization, as done in [Wang et al., 2017].
****************************************************************************

In [ ]:
# If sci span has split it, we are fine with it being split (except parentheses)